<a href="https://colab.research.google.com/github/kirtigupta25/Predictive_assignments/blob/main/Topsis_102117071.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
from transformers import pipeline
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import pairwise_distances


In [3]:
# List of prompts
prompts = [
    "Explain the concept of artificial intelligence.",
    "What are the benefits of regular exercise?",
    "Describe the process of photosynthesis.",
    "How does climate change impact biodiversity?",
    "Discuss the importance of cybersecurity in modern society."
]


In [8]:
# List of pre-trained models
models_to_compare = ["gpt2", "openai-gpt", "distilgpt2", "gpt2-medium"]

In [9]:
# Function to generate text using a specified model and prompt
def generate_text(model_name, prompt, max_length=50):
    generator = pipeline("text-generation", model=model_name)
    return generator(prompt, max_length=max_length, num_return_sequences=1)[0]['generated_text']

In [10]:
# Evaluation function
def evaluate_text_generation(model_name, prompt):
    # Generate text
    generated_text = generate_text(model_name, prompt)

    # Placeholder for simplicity
    perplexity = len(set(generated_text.split())) / len(generated_text.split())  # Unique words per total words
    coherence = len(set(zip(generated_text.split(), generated_text.split()[1:]))) / len(generated_text.split())  # Unique bigrams per total bigrams
    fluency = 1.0 / perplexity  # Placeholder, inversely proportional to perplexity
    relevance = 0.8  # Placeholder

    # Return a dictionary of evaluation scores
    evaluation_results = {
        'Perplexity': perplexity,
        'Coherence': coherence,
        'Fluency': fluency,
        'Relevance': relevance
    }

    return evaluation_results


In [11]:
# Evaluate models and calculate performance scores
criteria_data = []
for model_name in models_to_compare:
    model_scores = []
    for prompt in prompts:
        evaluation_results = evaluate_text_generation(model_name, prompt)
        model_scores.append(list(evaluation_results.values()))

    criteria_data.append(model_scores)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [15]:
criteria_data_flat = [item for sublist in criteria_data for item in sublist]
criteria_data_scaled = minmax_scale(criteria_data_flat, axis=0)


In [16]:
distances = pairwise_distances(criteria_data_scaled, metric='euclidean')
closeness = minmax_scale(distances, axis=0)
topsis_result = closeness.sum(axis=1)


In [18]:
print(len(prompts))
print(len(models_to_compare))
print(len(topsis_result))


5
4
20


In [23]:
import pandas as pd

# Repeat prompts for each model
all_prompts = [prompt for prompt in prompts for _ in models_to_compare]

# Create a DataFrame with the results
result_df = pd.DataFrame({'Prompt': all_prompts})

# Add columns for each model and corresponding TOPSIS result
for model_name, topsis_score in zip(models_to_compare * len(prompts), topsis_result):
    result_df[model_name] = topsis_score

# Add a column for the parameter used
result_df['Parameter'] = ['Perplexity', 'Coherence', 'Fluency', 'Relevance'] * len(prompts)

# Print the DataFrame
print(result_df)


                                               Prompt       gpt2  openai-gpt  \
0     Explain the concept of artificial intelligence.  11.588437    7.157371   
1     Explain the concept of artificial intelligence.  11.588437    7.157371   
2     Explain the concept of artificial intelligence.  11.588437    7.157371   
3     Explain the concept of artificial intelligence.  11.588437    7.157371   
4          What are the benefits of regular exercise?  11.588437    7.157371   
5          What are the benefits of regular exercise?  11.588437    7.157371   
6          What are the benefits of regular exercise?  11.588437    7.157371   
7          What are the benefits of regular exercise?  11.588437    7.157371   
8             Describe the process of photosynthesis.  11.588437    7.157371   
9             Describe the process of photosynthesis.  11.588437    7.157371   
10            Describe the process of photosynthesis.  11.588437    7.157371   
11            Describe the process of ph